In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import time
import pyttsx3
import os
import glob
import face_recognition
import tkinter as tk
from tkinter import simpledialog

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        images_path = glob.glob(os.path.join(images_path, "*.jpg"))

        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            face_encodings = face_recognition.face_encodings(rgb_img)

            if len(face_encodings) == 0:
                print(f"No faces found in {img_path}. Skipping...")
                continue

            img_encoding = face_encodings[0]

            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)


    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names

def detect_emotion(face_roi):
    final_image = cv2.resize(face_roi, (224, 224))
    final_image = np.expand_dims(final_image, axis=0)
    final_image = final_image / 255.0
    predictions = new_model.predict(final_image)
    emotion_label = emotion_labels[np.argmax(predictions)]
    return emotion_label

def speak_emotion_and_name(name, emotion):
    if name == "Unknown":
        engine.say(f"Unknown person is {emotion}")
    else:
        engine.say(f"{name} is {emotion}")
    engine.runAndWait()
    

def capture_screenshot(frame, save_path, filename):
    cv2.imwrite(os.path.join(save_path, filename), frame)
    print("Screenshot saved as:", filename)

def get_custom_name():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Create a simple dialog box to get user input
    custom_name = simpledialog.askstring("Custom Name", "Enter a custom name for the new face:")

    # Check if user clicked cancel or input is empty
    if custom_name is None or custom_name.strip() == "":
        print("No custom name provided.")
        return None

    return custom_name

path = "haarcascade_frontalface_default.xml"
font_scale = 1
font = cv2.FONT_HERSHEY_SIMPLEX
emotion_labels = ['Neutral', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

new_model = tf.keras.models.load_model('fer_model.h5')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
engine = pyttsx3.init()

IMAGES_FOLDER = "images"
sfr = SimpleFacerec()
sfr.load_encoding_images(IMAGES_FOLDER)

new_face_detected = True
emotion_last_spoken = time.time()

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        if not new_face_detected:
            cv2.putText(frame, "No face detected", (20, 50), font, font_scale, (255, 255, 255), 2, cv2.LINE_AA)
        new_face_detected = False
        cv2.imshow("Facial Recognition and Emotion Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        if new_face_detected:
            engine.say("New face detected")
            engine.runAndWait()
            new_face_detected = False

        emotion = detect_emotion(roi_color)
        face_locations, face_names = sfr.detect_known_faces(frame)
        

        for face_loc, name in zip(face_locations, face_names):
            y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
            cv2.putText(frame, name, (x1, y1 - 30), font, font_scale, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, emotion, (x1, y1 + h + 30), font, font_scale, (0, 255, 255), 2, cv2.LINE_AA)
            
            if time.time() - emotion_last_spoken > 5:
                
                emotion_last_spoken = time.time()
                speak_emotion_and_name(name, emotion)

    cv2.imshow("Facial Recognition and Emotion Recognition", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Press Esc to exit
        break
    elif key == ord('n'):  # Press 'n' to capture and save screenshot
        custom_name = get_custom_name()
        filename = custom_name + ".jpg"
        capture_screenshot(frame, IMAGES_FOLDER, filename)
        sfr.load_encoding_images(IMAGES_FOLDER)  # Reload encodings to include the new face

cap.release()
cv2.destroyAllWindows()
